<a href="https://colab.research.google.com/github/MalikHasnat1999/Deep-Learning/blob/master/Plant_Village.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# ! pip install -q kaggle

In [ ]:
# from google.colab import files
# files.upload()

In [ ]:
# Run this cell if you can't find the kaggle folder
# ! mkdir ~/.kaggle/    # make a kaggle folder in root directory

In [ ]:
# copy kaggle.json file to the kaggle folder in the root directory
# !cp kaggle.json ~/.kaggle/

In [ ]:
# Change the permissions of the .json file.
# !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# That's all ! You can check if everything's okay by running this command
# !kaggle datasets list 

In [ ]:
! git clone https://github.com/spMohanty/PlantVillage-Dataset

In [ ]:
! pip install split-folders

In [ ]:
import splitfolders
input_folder = "/content/PlantVillage-Dataset/raw/color"
splitfolders.ratio(input=input_folder,
                   output="plants_dataset",
                   ratio=(0.8,0.2),
                   seed=42)

In [ ]:
train_dir = "/content/plants_dataset/train" 
test_dir = "/content/plants_dataset/val" 

# Getting data from the Directory

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                    label_mode="categorical",
                                                                    image_size=(224,224))

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                   label_mode="categorical",
                                                                   image_size=(224,224),
                                                                   shuffle=False) # don't shuffle test data for prediction analysis

In [ ]:
# get class_names
class_names = train_dataset.class_names
class_names, len(class_names)

# Visualize the Data

In [ ]:
import random
import os
import matplotlib.image as mpimg


def random_img(target_dir, target_class):
  target_folder = target_dir + "/" + target_class
  rand_img = random.sample(os.listdir(target_folder), 8)
  plt.figure(figsize=(20,7))
  for i in range(8):
    img = mpimg.imread(target_folder + "/" + rand_img[i])
    plt.subplot(2,4,i+1)
    plt.imshow(img)
    plt.title(target_class)
    plt.axis("off")
    print(f"Shape: {img.shape}")

In [ ]:
random_img(train_dir, random.choice(class_names))

# Create Callbacks

Model Checkpoint

In [ ]:
import datetime
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath="model_checkpoint",
                                                      save_weights_only=True,
                                                      save_best_only=True,
                                                      monitor="val_accuracy")

# Create Augmentation Layer

In [ ]:
aug_layer = tf.keras.Sequential([
                tf.keras.layers.experimental.preprocessing.RandomZoom(0.2),
                tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
                tf.keras.layers.experimental.preprocessing.RandomHeight(0.2),
                tf.keras.layers.experimental.preprocessing.RandomWidth(0.2),
                tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
                #tf.keras.layers.experimental.preprocessing.Rescaling(scale=1/255.) # not needed for EfficientNet

], name="augmentation_layer")

# 1)Feature-Extraction

### (Without Augmentation)

In [ ]:
# set random seed
tf.random.set_seed(42)

# set up the base model
base_model = tf.keras.applications.EfficientNetB7(include_top=False)
base_model.trainable = False

# Create model with functional API
inputs = tf.keras.layers.Input(shape=(224,224,3), name="input_layer")
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_avg_pooling_layer")(x)
outputs = tf.keras.layers.Dense(len(class_names), activation="softmax", name="output_layer")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_featureExtraction_ENb7")

# compile the model
model_1.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics= ["accuracy"]
)

# fit the model
c = model_1.fit(train_dataset,
                              epochs=5,
                              steps_per_epoch=len(train_dataset),
                              validation_data=test_dataset,
                              validation_steps=int(0.15*len(test_dataset)),
                              callbacks=[model_checkpoint])

In [ ]:
model_1.summary()

In [ ]:
model_1_results = model_1.evaluate(test_dataset)
model_1_results

### (With Augmentation)

In [ ]:
# set random seed
tf.random.set_seed(42)

# set up the base model
base_model = tf.keras.applications.EfficientNetB7(include_top=False)
base_model.trainable = False

# create model with functional API
inputs = tf.keras.layers.Input(shape=(224,224,3), name="input_layer")
x = aug_layer(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_avg_pooling_layer")(x)
outputs = tf.keras.layers.Dense(len(class_names), activation="softmax", name="output_layer")(x)
model_2 = tf.keras.Model(inputs, outputs)

# compile the model
model_2.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"]
)

# fit the model
model_2_history = model_2.fit(train_dataset,
                              epochs=5,
                              steps_per_epoch=len(train_dataset),
                              validation_data=test_dataset,
                              validation_steps=int(0.15*(len(test_dataset))),
                              callbacks=[model_checkpoint])

In [ ]:
model_2.summary()

In [ ]:
model_2_results = model_2.evaluate(test_dataset)
model_2_results

# 2)Fine-Tuning

###(Without Augmentation)

In [ ]:
# set up base model
base_model = tf.keras.applications.EfficientNetB7(include_top=False)
# unfreeze all layers
base_model.trainable = True

# freeze all layers except for last 10
for layer in base_model.layers[:-10]:
  layer.trainable = False

In [ ]:
for i, layer in enumerate(base_model.layers):
  print(i,layer.name,layer.trainable)

In [ ]:
# set random seed 
tf.random.set_seed(42)

# create model using Functional API
inputs = tf.keras.layers.Input(shape=(224,224,3), name="input_layer")
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(len(class_names), activation="softmax", name="output_layer")(x)
model_3 = tf.keras.Model(inputs, outputs)

# compile the model
model_3.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(lr=0.0001),
    metrics=["accuracy"]
)

# fit the model
model_3_history = model_3.fit(train_dataset,
                              epochs=10,
                              steps_per_epoch=len(train_dataset),
                              validation_data=test_dataset,
                              validation_steps=int(0.15*len(test_dataset)),
                              initial_epoch=model_1_history.epoch[-1])

In [ ]:
model_3.evaluate(test_dataset)

### (With Augmentation)

In [ ]:
# model with functional API
inputs = tf.keras.layers.Input(shape=(224,224,3), name="input_shape")
x = aug_layer(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(len(class_names), activation="softmax")(x)
model_4 = tf.keras.Model(inputs, outputs)

# compile the model
model_4.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(lr=0.0001),
    metrics=["accuracy"]
)

# fit the model
model_4_history = model_4.fit(train_dataset,
                              epochs=10,
                              steps_per_epoch=len(train_dataset),
                              validation_data=test_dataset,
                              validation_steps=int(0.15*len(test_dataset)),
                              initial_epoch=model_2_history.epoch[-1])

In [ ]:
model_4.evaluate(test_dataset)

In [ ]:
model_3.save("/content/drive/MyDrive/Saved_Models/PlantVillage-Dataset/Fine-tune-without-augmentation/ENb7")

In [ ]:
model_4.save("/content/drive/MyDrive/Saved_Models/PlantVillage-Dataset/Fine-tune-with-augmentation/ENb7.h5")

In [ ]:
loaded_model = tf.keras.models.load_model("/content/drive/MyDrive/Saved_Models/PlantVillage-Dataset/Fine-tune-with-augmentation/ENb7.h5")

In [ ]:
loaded_model.evaluate(test_dataset)

In [ ]:
loaded_model_1 = tf.keras.models.load_model("/content/drive/MyDrive/Saved_Models/PlantVillage-Dataset/Fine-tune-without-augmentation/ENb7")

In [ ]:
loaded_model_1.evaluate(test_dataset)

# 3)Scaling-up





## Evaluating the performance of our model across all different classes

In [ ]:
# Check to see if loaded model is a trained model
loaded_loss, loaded_accuracy = loaded_model_1.evaluate(test_dataset)
loaded_loss, loaded_accuracy

In [ ]:
# Make predictions with model
pred_probs = loaded_model_1.predict(test_dataset, verbose=1) # set verbosity to see how long it will take 

In [ ]:
# How many predictions are there?
len(pred_probs)

In [ ]:
# What's the shape of our predictions?
pred_probs.shape

In [ ]:
# How do they look?
pred_probs[:10]

In [ ]:
# We get one prediction probability per class
print(f"Number of prediction probabilities for sample 0: {len(pred_probs[0])}")
print(f"What prediction probability sample 0 looks like:\n {pred_probs[0]}")
print(f"The class with the highest predicted probability by the model for sample 0: {pred_probs[0].argmax()}")

In [ ]:
# Get the class predicitons of each label
y_pred = pred_probs.argmax(axis=1)
# How do they look?
y_pred[:10]

In [ ]:
# Note: This might take a minute or so due to unravelling 790 batches
y_labels = []
for images, labels in test_dataset.unbatch(): # unbatch the test data and get images and labels
  y_labels.append(labels.numpy().argmax()) # append the index which has the largest value (labels are one-hot)
y_labels[:10] # check what they look like (unshuffled)

## Evaluating our models predictions

### accuracy_score()

In [ ]:
from sklearn.metrics import accuracy_score
sklearn_accuracy = accuracy_score(y_labels, y_pred)
sklearn_accuracy

### confusion_matrix()

In [ ]:
# Note: The following confusion matrix code is a remix of Scikit-Learn's 
# plot_confusion_matrix function - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Our function needs a different name to sklearn's plot_confusion_matrix
def plot_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False): 
  """Makes a labelled confusion matrix comparing predictions and ground truth labels.

  If classes is passed, confusion matrix will be labelled, if not, integer class values
  will be used.

  Args:
    y_true: Array of truth labels (must be same shape as y_pred).
    y_pred: Array of predicted labels (must be same shape as y_true).
    classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
    figsize: Size of output figure (default=(10, 10)).
    text_size: Size of output figure text (default=15).
    norm: normalize values or not (default=False).
    savefig: save confusion matrix to file (default=False).
  
  Returns:
    A labelled confusion matrix plot comparing y_true and y_pred.

  Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
  """  
  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])
  
  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes), 
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)
  
  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  ### Added: Rotate xticks for readability & increase font size (required due to such a large confusion matrix)
  plt.xticks(rotation=70, fontsize=text_size)
  plt.yticks(fontsize=text_size)

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  # Save the figure to the current working directory
  if savefig:
    fig.savefig("confusion_matrix.png")

In [ ]:
plot_confusion_matrix(y_labels, y_pred, classes=class_names, figsize=(100,100))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_labels, y_pred))

In [ ]:
classification_report_dict = classification_report(y_labels, y_pred, output_dict=True)
classification_report_dict

In [ ]:
# Create empty dictionary
class_f1_scores = {}
# Loop through classification report items
for k, v in classification_report_dict.items():
  if k == "accuracy": # stop once we get to accuracy key
    break
  else:
    # Append class names and f1-scores to new dictionary
    class_f1_scores[class_names[int(k)]] = v["f1-score"]
class_f1_scores

In [ ]:
# Turn f1-scores into dataframe for visualization
import pandas as pd
f1_scores = pd.DataFrame({"class_name": list(class_f1_scores.keys()),
                          "f1-score": list(class_f1_scores.values())}).sort_values("f1-score", ascending=False)
f1_scores

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 25))
scores = ax.barh(range(len(f1_scores)), f1_scores["f1-score"].values)
ax.set_yticks(range(len(f1_scores)))
ax.set_yticklabels(list(f1_scores["class_name"]))
ax.set_xlabel("f1-score")
ax.set_title("F1-Scores for 10 Different Classes")
ax.invert_yaxis(); # reverse the order

def autolabel(rects): # Modified version of: https://matplotlib.org/examples/api/barchart_demo.html
  """
  Attach a text label above each bar displaying its height (it's value).
  """
  for rect in rects:
    width = rect.get_width()
    ax.text(1.03*width, rect.get_y() + rect.get_height()/1.5,
            f"{width:.2f}",
            ha='center', va='bottom')

autolabel(scores)